In [15]:
#https://www.kaggle.com/c/word2vec-nlp-tutorial
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from nltk.stem.porter import PorterStemmer

import utils
import importlib
importlib.reload(utils)

<module 'utils' from '/home/andryw/Projects/tutorials/embeddings/utils.py'>

In [13]:
data = pd.read_csv("data/labeledTrainData.tsv", delimiter="\t", quoting=3)
train_data, test_data = train_test_split(data, test_size=0.25)

In [16]:
# Clean the text
train_data['review_cleaned'] = train_data.review.apply(lambda x: utils.normalize_text(x, to_stem=True))
test_data['review_cleaned'] = test_data.review.apply(lambda x: utils.normalize_text(x , to_stem=True))

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# # Get bag of words from the text
# count_vectorizer = CountVectorizer(max_features=6000)
# train_bow = count_vectorizer.fit_transform(list(train_data['review_cleaned']) + 
#                                            list(test_data['review_cleaned']))

# train_bow = count_vectorizer.transform(train_data['review_cleaned']).toarray()
# print("dimentions train: {}".format(train_bow.shape))

# test_bow = count_vectorizer.transform(test_data['review_cleaned']).toarray()
# print("dimentions of test data: {}".format(test_bow.shape))

In [50]:
# Get tfidf from the text
vectorizer = TfidfVectorizer(max_features=10000)
train_bow = vectorizer.fit_transform(list(train_data['review_cleaned']) + 
                                           list(test_data['review_cleaned']))
train_bow = vectorizer.transform(train_data['review_cleaned']).toarray()
print("dimentions: {}".format(train_bow.shape))

test_bow = vectorizer.transform(test_data['review_cleaned']).toarray()
print("dimentions of test data: {}".format(test_bow.shape))

dimentions: (18750, 10000)
dimentions of test data: (6250, 10000)


In [51]:
x_train = train_bow
y_train = train_data.sentiment

x_test = test_bow
y_test  = test_data.sentiment

## Classifier

### XGboost

In [26]:
# params = {"n_jobs": 3, "silent": True, "n_estimators": 100, 'objective':'binary:logistic'}
# xgboost_classifier = xgb.XGBClassifier(**params)

# xgboost_classifier.fit(x_train, y_train, verbose=True, eval_metric='auc',
#                       eval_set=[(x_train, y_train), (x_test, y_test)],)

# metrics.roc_auc_score(xgboost_classifier.predict(x_test), y_test)

### RandomForest

In [27]:
random_forest = RandomForestClassifier(n_estimators=100, verbose=True, n_jobs=3)
random_forest = random_forest.fit(x_train, y_train)
metrics.roc_auc_score(random_forest.predict(x_test), y_test)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   16.9s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:   39.5s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.4s finished


0.8392129890990521

### Logistc

In [52]:
logistic_regression = LogisticRegression(verbose=True, n_jobs=3)
logistic_regression = logistic_regression.fit(x_train, y_train)
metrics.roc_auc_score(logistic_regression.predict(x_test), y_test)

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]

0.8797813686575502

## Predict

In [39]:
unlabel_data = pd.read_csv("data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabel_data['review_cleaned'] = unlabel_data.review.apply(lambda x: utils.normalize_text(x))

In [20]:
utils.save_predict(unlabel_data, 
                   random_forest.predict(unlabel_data_bow), 
                   "random_forest_tfidf.csv")

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.4s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.9s finished


In [21]:
utils.save_predict(unlabel_data, 
                   xgboost_classifier.predict(unlabel_data_bow), 
                   "xgboost_tfidf.csv")

In [53]:
logistic_regression = LogisticRegression(verbose=True, n_jobs=3)
logistic_regression = logistic_regression.fit(np.concatenate((x_train, x_test), axis=0), 
                                              np.concatenate((y_train, y_test), axis=0))

unlabel_data_bow = vectorizer.transform(unlabel_data['review_cleaned'])
unlabel_data_bow = unlabel_data_bow.toarray()

utils.save_predict(unlabel_data, 
             logistic_regression.predict(unlabel_data_bow), 
             "regression_tfidf_stem_10000-features.csv")

/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/andryw/Projects/tutorials/python_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[LibLinear]